# LeCun et al. 1990, "Handwritten Digit Recognition: Applications of Neural Net Chips and Automatic Learning"
> The following tries to reproduce the original paper. Note that the digits dataset actually used in the paper could not be found and [MNIST 784](https://www.openml.org/search?type=data&status=active&id=554) is used instead

## References

* LeCun et al. 1990, _Handwritten Digit Recognition: Applications of Neural Net Chips and Automatic Learning_, [Neurocomputing](https://link.springer.com/chapter/10.1007/978-3-642-76153-9_35)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random
import typing as T
from collections import defaultdict

# plot first item in dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
from einops import rearrange
from einops.layers.torch import Rearrange
from sklearn import metrics

# load mnist using scikit-learn
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from torch.optim import SGD
from torch.utils.data import DataLoader, Dataset

sns.set_theme()

specifics in the paper:

* neural net
    * weight initialization: uniformly at random $\in [-2.4 / F_i, 2.4 / F_i]$ with $F_i = $ number of inputs of the unit
    * "tanh activation": $A \cdot \tanh (S \cdot a)$ with $A = 1.716$, $S = 2/3$ and $a = \text{weights} \cdot \text{input}$
    * 256 input (16 x 16 pixel images)
    * layer #1: 
        * convolution with 12 5x5-kernels and stride 2 (output: 8 x 8 x 12 = 786 "units")
        * tanh activation
        * $F_i = 5 \cdot 5 \cdot n_\text{input-channels} = 5 \cdot 5 \cdot 1 = 25$
    * layer #2: 
        * convolution with 12 5x5-kernels and stride 2 (output: 4 x 4 x 12 = 192 "units")
        * tanh activation
        * $F_i = 5 \cdot 5 \cdot n_\text{input-channels} = 5 \cdot 5 \cdot 12 = 300$
    * layer #3:
        * dense with 30 neurons
        * tanh activation
        * $F_i = 4 \cdot 4 \cdot 12 = 192$
    * layer #4:
        * dense output layer with 10 neurons
        * tanh activation
        * $F_i = 30$
* target: vector of 10 values either 1 or -1 (so 9x -1 and 1x 1)
* loss: mean squared error between prediction and target (paper reached 1.8e-2 on test and 2.5e-3 on train)
* error rates: 0.14% on train, 5% on test
* training:
    * stochastic gradient descent (1 sample per backpropagation)
    * samples always in the same order, no shuffling
    * 23 or 30 epochs, paper is ambiguous
    * learning rate was set using some not defined 2nd order derivative method

In [ ]:
mnist = fetch_openml("mnist_784", version=1, cache=True, parser="auto")

Setting seeds

In [ ]:
torch.manual_seed(42)

random.seed(42)

np.random.seed(42)

In [ ]:
def get_device() -> str:
    return "cuda" if torch.cuda.is_available() else "cpu"


device = get_device()
device

In [ ]:
X = mnist["data"]
y = mnist["target"]
X.shape, y.shape

In [ ]:
n = 100
X0, y0 = X.iloc[:n], y.iloc[:n]

## Defining dataset and dataloader

In [ ]:
class DigitsDataset(Dataset):
    def __init__(self, X: pd.DataFrame, y: pd.Series, edge: int = 28):
        self.X = X
        self.y = y
        self.edge = edge

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx: int) -> T.Tuple[torch.Tensor, int]:
        img = (
            torch.from_numpy(self.X.iloc[idx].values / 255.0)  # normalizing
            .reshape(self.edge, self.edge)
            .double()
        )
        label = int(self.y.iloc[idx])
        return (img, label)

In [ ]:
ds = DigitsDataset(X0, y0)

In [ ]:
item = ds[4]
plt.imshow(item[0], cmap="gray", origin="upper")
plt.title(f"Label: {item[1]}")
plt.tight_layout()

In [ ]:
batch_size = 1
dataloader = DataLoader(ds, batch_size=batch_size, shuffle=False)

In [ ]:
train_features, train_labels = next(iter(dataloader))

In [ ]:
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0]  # .reshape((28,28))
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [ ]:
def calc_conv_output_dim(input_dim, kernel_size, padding, stride):
    return int((input_dim - kernel_size + 2 * padding) / stride + 1)


calc_conv_output_dim(28, 5, 2, 2), calc_conv_output_dim(14, 5, 2, 2)

## `MyConv2d`

In [ ]:
class MyConv2d(torch.nn.Module):
    def __init__(
        self,
        edge: int,
        n_in_channels: int = 1,
        n_out_channels: int = 1,
        kernel_width: int = 5,
        kernel_height: int = 5,
        stride: int = 1,
        padding: int = 0,
        dilation: int = 1,
        lecun_init: bool = True,
    ):
        super().__init__()

        self.register_buffer("edge", torch.tensor(edge))
        self.register_buffer("n_in_channels", torch.tensor(n_in_channels))
        self.register_buffer("n_out_channels", torch.tensor(n_out_channels))
        self.register_buffer("kernel_width", torch.tensor(kernel_width))
        self.register_buffer("kernel_height", torch.tensor(kernel_height))
        self.register_buffer("stride", torch.tensor(stride))
        self.register_buffer("padding", torch.tensor(padding))
        self.register_buffer("dilation", torch.tensor(dilation))

        self.weight = nn.Parameter(
            torch.empty(
                n_in_channels * kernel_width * kernel_height,
                n_out_channels,
                dtype=torch.double,
            )
        )
        self.bias = nn.Parameter(
            torch.empty(1, n_out_channels, 1, 1, dtype=torch.double)
        )

        # self.bias = rearrange(self.bias, "out_channels -> 1 out_channels 1 1")

        if lecun_init:
            s = 2.4 / (n_in_channels * kernel_width * kernel_height)
            self.weight.data.uniform_(-s, s)
            self.bias.data.uniform_(-s, s)

        else:
            self.weight.data.normal_(0, 1.0)
            self.bias.data.normal_(0, 1.0)

        self.unfold = torch.nn.Unfold(
            kernel_size=(kernel_height, kernel_width),
            dilation=dilation,
            padding=padding,
            stride=stride,
        )
        out_h = out_w = calc_conv_output_dim(
            edge, kernel_width, padding, stride
        )
        self.fold = torch.nn.Fold(
            output_size=(out_h, out_w),
            kernel_size=(1, 1),
            dilation=dilation,
            padding=0,
            stride=1,
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        # inspired by: https://discuss.pytorch.org/t/make-custom-conv2d-layer-efficient-wrt-speed-and-memory/70175/2

        # (N,C,in_h,in_w) -> (N, C*kh*kw, num_patches)
        # N = batch_size, C = in_channels, kh = kernel_height, kw = kernel_width
        input_unfolded = self.unfold(input)

        input_unfolded = rearrange(
            input_unfolded, "N r num_patches -> N num_patches r"
        )

        output_unfolded = input_unfolded @ self.weight
        output_unfolded = rearrange(
            output_unfolded,
            "N num_patches out_channels -> N out_channels num_patches",
        )

        output = self.fold(output_unfolded)  # (N, out_channels, out_h, out_w)
        if self.bias is not None:
            output += self.bias

        return output


kh = kw = 5
n_in_channels = 1
n_out_channels = 1
weight = torch.randn(n_out_channels, n_in_channels, kw, kw).double()
bias = torch.randn(n_out_channels).double()

print(f"{weight.shape=}")

train_features, train_labels = next(iter(dataloader))
train_features = train_features.unsqueeze(dim=1)

print(f"{train_features.shape=}")

myconv2d = MyConv2d(
    edge=28,
    n_in_channels=n_in_channels,
    n_out_channels=n_out_channels,
    kernel_width=kw,
    kernel_height=kh,
    stride=2,
    padding=2,
    dilation=1,
)
conv_features = myconv2d(train_features)

print(f"{conv_features.shape=}")

In [ ]:
label = train_labels[0]
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(5, 5))

ax = axs[0]
img = train_features[0][0]
ax.imshow(img, cmap="gray")

ax = axs[1]
img = conv_features.detach().numpy()[0][0]
ax.imshow(img, cmap="gray")

plt.show()
print(f"Label: {label}")

### just trying to overfit

In [ ]:
class ParameterHistory:
    def __init__(self, every_n: int = 1):
        self.history = defaultdict(list)
        self.every_n = every_n
        self.iter = []

    def __call__(self, model: nn.Module, _iter: int):
        if _iter % self.every_n != 0:
            return
        state_dict = model.state_dict()

        for name, tensor in state_dict.items():
            self.history[name].append(tensor.clone().numpy().ravel())

        self.iter.append(_iter)

    def get_df(self, name: str) -> pd.DataFrame:
        df = [
            pd.DataFrame({"value": w}).assign(iter=i)
            for i, w in zip(self.iter, self.history[name])
        ]
        return pd.concat(df, ignore_index=True)[["iter", "value"]]

    def get_rolling_mean_df(self, name: str, window: int = 10) -> pd.DataFrame:
        df = self.get_df(name)
        df_roll = df.rolling(window=window, on="iter", min_periods=1).mean()
        if "iter" not in df_roll.columns:
            df_roll["iter"] = range(len(df_roll))
        return df_roll


class LossHistory:
    def __init__(self, every_n: int = 1):
        self.history = []
        self.iter = []
        self.every_n = every_n

    def __call__(self, loss: torch.Tensor, _iter: int):
        if _iter % self.every_n != 0:
            return
        self.history.append(loss.item())
        self.iter.append(_iter)

    def get_df(self) -> pd.DataFrame:
        return pd.DataFrame({"iter": self.iter, "loss": self.history})

    def get_rolling_mean_df(self, window: int = 10) -> pd.DataFrame:
        df = self.get_df()
        df_roll = df.rolling(window=window, on="iter", min_periods=1).mean()
        if "iter" not in df_roll.columns:
            df_roll["iter"] = range(len(df_roll))
        return df_roll

In [ ]:
def densify_y(y: torch.Tensor) -> torch.Tensor:
    new_y = F.one_hot(y, num_classes=10)
    new_y[new_y == 0] = -1
    return new_y.double()


train_labels[0:3], densify_y(train_labels[0:3])

In [ ]:
class TanhLeCun1990(nn.Module):
    def __init__(self, A: float = 1.716, S: float = 2 / 3):
        super().__init__()
        self.A = A
        self.S = S

    def forward(self, x: torch.Tensor):
        return self.A * torch.tanh(self.S * x)


class Model(nn.Module):
    # inspired by https://einops.rocks/pytorch-examples.html
    def __init__(
        self,
        edge: int = 28,
        n_classes: int = 10,
        lecun_init: bool = True,
        lecun_act: bool = True,
    ):
        super().__init__()

        # self.conv1 = nn.Conv2d(1, 12, kernel_size=5, stride=2, padding=2)
        self.conv1 = MyConv2d(
            edge=edge,
            n_in_channels=1,
            n_out_channels=12,
            kernel_width=5,
            kernel_height=5,
            stride=2,
            padding=2,
            lecun_init=lecun_init,
        )
        edge = edge // 2  # effect of stride
        # self.conv2 = nn.Conv2d(12, 12, kernel_size=5, stride=2, padding=2)
        self.conv2 = MyConv2d(
            edge=edge,
            n_in_channels=12,
            n_out_channels=12,
            kernel_width=5,
            kernel_height=5,
            stride=2,
            padding=2,
            lecun_init=lecun_init,
        )
        edge = edge // 2  # effect of stride
        self.lin1 = nn.Linear(edge * edge * 12, 30)
        self.lin2 = nn.Linear(30, n_classes)

        if lecun_init:
            s = 2.4 / self.lin1.weight.shape[0]
            self.lin1.weight.data.uniform_(-s, s)

            s = 2.4 / self.lin2.weight.shape[0]
            self.lin2.weight.data.uniform_(-s, s)

        if lecun_act:
            self.act = TanhLeCun1990()
        else:
            self.act = F.tanh

        self.net = nn.Sequential(
            Rearrange("b h w -> b 1 h w"),
            self.conv1,
            self.act,
            self.conv2,
            self.act,
            Rearrange("b c h w -> b (c h w)"),
            self.lin1,
            self.act,
            self.lin2,
            self.act,
        )

    def forward(self, x: torch.Tensor):
        return self.net(x)

In [ ]:
model = Model(lecun_init=True, lecun_act=True)
model.double()

In [ ]:
opt = SGD(
    model.parameters(),
    lr=0.1,  # randomly chosen, not provided in the paper
)

In [ ]:
# loss_func = nn.CrossEntropyLoss()
loss_func = nn.MSELoss()

In [ ]:
model.to(device);

In [ ]:
parameter_history = ParameterHistory(every_n=100)
loss_history = LossHistory(every_n=2)

In [ ]:
n_epochs = 20
_iter = 0
model.train()
for epoch in tqdm.tqdm(range(n_epochs), desc="Epochs", total=n_epochs):
    for i, (xb, yb) in tqdm.tqdm(
        enumerate(dataloader), desc="Batches", total=len(dataloader)
    ):
        xb = xb.to(device)
        yb = yb.to(device)
        yb = densify_y(yb)
        loss = loss_func(model(xb), yb)

        opt.zero_grad()
        loss.backward()
        opt.step()

        parameter_history(model, _iter)
        loss_history(loss, _iter)

        _iter += 1

print("Done!")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4), sharex=True)

df = loss_history.get_df()
df_roll = loss_history.get_rolling_mean_df(window=10)

sns.lineplot(data=df, x="iter", y="loss", ax=ax, label="Train")
sns.lineplot(
    data=df_roll, x="iter", y="loss", ax=ax, label="Train (rolling mean)"
)
ax.set(xlabel="Iter", ylabel="Loss", title="Loss History")

plt.tight_layout()

display(df_roll.tail())

In [ ]:
def draw_history(
    history: ParameterHistory,
    name: str,
    figsize: T.Tuple[int, int] = (12, 4),
    weight_bins: int = 20,
    bias_bins: int = 10,
) -> None:
    fig, axs = plt.subplots(figsize=figsize, nrows=2, sharex=True)

    ax = axs[0]
    _name = f"{name}.weight"
    df = history.get_df(_name)

    n_iter = df["iter"].nunique()
    bins = (n_iter, weight_bins)
    sns.histplot(
        data=df,
        x="iter",
        y="value",
        ax=ax,
        thresh=None,
        cmap="plasma",
        bins=bins,
    )
    ax.set_ylabel("weight")
    ax.set_title(name)

    ax = axs[1]
    _name = f"{name}.bias"
    df = history.get_df(_name)

    bins = (n_iter, bias_bins)
    sns.histplot(
        data=df,
        x="iter",
        y="value",
        ax=ax,
        thresh=None,
        cmap="plasma",
        bins=bins,
    )
    ax.set_xlabel("iter")
    ax.set_ylabel("bias")

    plt.tight_layout()
    plt.show()


draw_history(parameter_history, "conv1")
draw_history(parameter_history, "conv2")
draw_history(parameter_history, "lin1", weight_bins=100)
draw_history(parameter_history, "lin2")

In [ ]:
train_features, train_labels = next(iter(dataloader))

In [ ]:
model.eval();

In [ ]:
train_features = train_features.to(device)
pred_probs = model(train_features)
pred_probs

In [ ]:
y_pred = pred_probs.to("cpu").detach().numpy().argmax(axis=1)
y_pred

In [ ]:
train_labels

In [ ]:
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].cpu()  # .reshape((28,28))
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}, pred: {y_pred[0]}")

In [ ]:
list(model.conv1.buffers())

In [ ]:
n_filters = model.conv1.n_out_channels.item()

fig, axs = plt.subplots(
    nrows=n_filters // 3, ncols=n_filters // 4, figsize=(12, 12)
)
with torch.no_grad():
    conv_features = model.act(model.conv1(train_features.unsqueeze(1)))
    for i, ax in enumerate(axs.flatten()):
        ax.imshow(conv_features[0][i], cmap="gray")
        ax.axis("off")
        ax.set_title(f"Filter {i+1}")

plt.show()
print(f"Label: {label}")

### Getting decent performance on 80:20 split over 10k digits

In [ ]:
n = 10_000  # bit larger than the number of samples in the paper
X0, X1, y0, y1 = train_test_split(
    X.iloc[:n], y.iloc[:n], test_size=0.2, random_state=42
)  # , stratify=y)

In [ ]:
ds = DigitsDataset(X0, y0)
ds_test = DigitsDataset(X1, y1)

In [ ]:
batch_size = 1
dataloader = DataLoader(ds, batch_size=batch_size, shuffle=False)
dataloader_test = DataLoader(ds_test, batch_size=500, shuffle=False)

In [ ]:
model = Model(lecun_init=True, lecun_act=True)
model.double()

In [ ]:
opt = SGD(
    model.parameters(),
    lr=0.01,  # randomly chosen, not provided in the paper
)

In [ ]:
# loss_func = nn.CrossEntropyLoss()
loss_func = nn.MSELoss()

In [ ]:
model.to(device);

In [ ]:
parameter_history = ParameterHistory(every_n=100)
loss_history = LossHistory(every_n=100)
loss_history_test = LossHistory(every_n=100)

In [ ]:
n_epochs = 10
_iter = 0
model.train()
for epoch in tqdm.tqdm(range(n_epochs), desc="Epochs", total=n_epochs):
    for i, (xb, yb) in tqdm.tqdm(
        enumerate(dataloader), desc="Batches", total=len(dataloader)
    ):
        xb = xb.to(device)
        yb = yb.to(device)
        yb = densify_y(yb)
        loss = loss_func(model(xb), yb)

        opt.zero_grad()
        loss.backward()
        opt.step()

        parameter_history(model, _iter)
        loss_history(loss, _iter)

        _iter += 1

    # compute validation loss
    with torch.no_grad():
        model.eval()
        ys_pred, ys_true = [], []
        for xb, yb in dataloader_test:
            xb = xb.to(device)
            yb = yb.to(device)
            yb = densify_y(yb)
            yp = model(xb)
            ys_pred.append(yp)
            ys_true.append(yb)
        y_pred = torch.cat(ys_pred, dim=0)
        y_true = torch.cat(ys_true, dim=0)
        loss_test = loss_func(y_pred, y_true)
        loss_history_test(loss_test, _iter)
        model.train()

print("Done!")

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(12, 4), sharex=True)

ax = axs[0]
df = loss_history.get_df()
df_roll = loss_history.get_rolling_mean_df(window=10)

sns.lineplot(data=df, x="iter", y="loss", ax=ax, label="Train")
sns.lineplot(
    data=df_roll, x="iter", y="loss", ax=ax, label="Train (rolling mean)"
)
ax.set(xlabel="Iter", ylabel="Loss", title="Loss History")

ax = axs[1]
df = loss_history_test.get_df()
df_roll = loss_history_test.get_rolling_mean_df(window=10)

sns.lineplot(data=df, x="iter", y="loss", ax=ax, label="Test")
sns.lineplot(
    data=df_roll, x="iter", y="loss", ax=ax, label="Test (rolling mean)"
)
ax.set(xlabel="Iter", ylabel="Loss", title="Test Loss History")

plt.tight_layout()

display(df_roll.tail())

In [ ]:
draw_history(parameter_history, "conv1")
draw_history(parameter_history, "conv2")
draw_history(parameter_history, "lin1", weight_bins=100)
draw_history(parameter_history, "lin2")

In [ ]:
train_features, train_labels = next(iter(dataloader))

In [ ]:
model.eval();

In [ ]:
train_features = train_features.to(device)
pred_probs = model(train_features)
pred_probs

In [ ]:
y_pred = pred_probs.to("cpu").detach().numpy().argmax(axis=1)
y_pred

In [ ]:
train_labels

In [ ]:
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].cpu()  # .reshape((28,28))
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}, pred: {y_pred[0]}")

In [ ]:
n_filters = model.conv1.n_out_channels.item()

fig, axs = plt.subplots(
    nrows=n_filters // 3, ncols=n_filters // 4, figsize=(12, 12)
)
with torch.no_grad():
    conv_features = model.act(model.conv1(train_features.unsqueeze(1)))
    for i, ax in enumerate(axs.flatten()):
        ax.imshow(conv_features[0][i], cmap="gray")
        ax.axis("off")
        ax.set_title(f"Filter {i+1}")

plt.show()
print(f"Label: {label}")

In [ ]:
ys_pred = []
ys_true = []
for test_features, test_labels in dataloader_test:
    test_features = test_features.to(device)
    pred_probs = model(test_features)

    y_pred = pred_probs.to("cpu").detach().numpy().argmax(axis=1)

    ys_true.append(test_labels.numpy())
    ys_pred.append(y_pred)

ys_true = np.concatenate(ys_true)
ys_pred = np.concatenate(ys_pred)

In [ ]:
ys_true, ys_pred

In [ ]:
accuracy = metrics.accuracy_score(ys_true, ys_pred)
error_rate = 1 - accuracy
print(f"Accuracy: {accuracy:.2%}")
print(f"Error rate: {error_rate:.2%}")

In [ ]:
ax = metrics.ConfusionMatrixDisplay.from_predictions(ys_true, ys_pred)
plt.show()